In [1]:
import pandas as pd
import numpy as np
import warnings
import boto3
from tqdm import tqdm
warnings.filterwarnings("ignore")

### data

In [2]:
client = boto3.client('s3')

In [3]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

In [4]:
response = client.list_objects_v2(
    Bucket=bucket,
    Prefix=f'{primary_folder}/latest_data/feature_prep/')

In [5]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    if '_base' in content['Key'] or '_diff' in content['Key']:
        paths.append(f"{s3_prefix}{bucket}/{content['Key']}")

In [6]:
paths

['s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_diff.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10D_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10D_diff.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10min_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10min_diff.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_120min_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_120min_diff.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_150D_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_b

In [7]:
def calculate_rsi(data, window=14):
    # Calculate price differences
    delta = data.diff()
    # Make two series: one for gains and one for losses
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)

    # Calculate the Exponential Moving Average of gains and losses
    avg_gain = gain.ewm(alpha=1/window, min_periods=window, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/window, min_periods=window, adjust=False).mean()

    # Calculate the RSI
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def read_and_calculate_rsi(path, windows, fields):
    
    print(f'Reading from {path}')
    df = pd.read_parquet(path)[fields + ['symbol']]
    grouped = df.groupby('symbol')
    results = []
    for symbol, group in grouped:
        calculated_fields = []
        for col in fields:
            for window in windows:
                rsi_values = calculate_rsi(group[col], window)
                group[f'{col}_{window}window_rsi'] = rsi_values
                calculated_fields.append(f'{col}_{window}window_rsi')
        results.append(group[fields + calculated_fields])
        # Concatenate all the grouped results back into a single DataFrame
    df_concat = pd.concat(results)
    loc = path.replace('.parquet', '_rsi.parquet')
    print(f'Saving to {loc}')
    df_concat.to_parquet(loc)
    del df, grouped, group, rsi_values, results, df_concat

In [8]:
for path in tqdm(paths):
    name = path.split('/')[-1].split('.')[0].split('_')[-2]
    if 'diff' in path:
        continue
    # if 'min' in name and int(name.replace('min', '')) < 100:  # Process files for <100 min
    print(path, '-', name)
    if 'min' in name:
        if int(name.replace('min', ''))<=10:
            windows = [7, 9, 11, 14, 17, 21]
            fields=['close', 'close_ema_2m', 'close_sma_2m']
        elif int(name.replace('min', ''))<=30:
            windows = [7, 9, 11, 14, 17, 21]
            fields=['close', 'close_ema_2m', 'close_sma_2m', 'close_ema_3m', 'close_sma_3m', 'close_ema_5m', 'close_sma_5m']
        else:
            windows = [7, 9, 11, 14, 17, 21]
            fields=['close', 'close_ema_5m', 'close_sma_5m', 'close_ema_10m', 'close_sma_10m', 'close_ema_20m', 'close_sma_20m', 'close_ema_50m', 'close_sma_50m']
    elif 'D' in name:
        if int(name.replace('D', '')) < 100:
            windows = [5, 7, 9, 11, 14, 17, 21]
            fields=['close', 'close_ema_5m', 'close_sma_5m', 'close_ema_10m', 'close_sma_10m', 'close_ema_20m', 'close_sma_20m', 'close_ema_50m', 'close_sma_50m']
        else: 
            windows = [5, 7, 9, 11, 14, 17, 21]
            fields=['close', 'close_ema_5m', 'close_sma_5m', 'close_ema_10m', 'close_sma_10m', 'close_ema_20m', 'close_sma_20m', 'close_ema_50m', 'close_sma_50m']
    read_and_calculate_rsi(path, windows, fields)

  0%|          | 0/50 [00:00<?, ?it/s]

s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_base.parquet - 100D
Reading from s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_base.parquet
Saving to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_base_rsi.parquet


  2%|▏         | 1/50 [00:01<01:11,  1.47s/it]

s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10D_base.parquet - 10D
Reading from s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10D_base.parquet
Saving to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10D_base_rsi.parquet


  6%|▌         | 3/50 [00:02<00:41,  1.13it/s]

...

 90%|█████████ | 45/50 [01:29<00:09,  1.89s/it]

s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_5min_base.parquet - 5min
Reading from s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_5min_base.parquet
Saving to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_5min_base_rsi.parquet


 94%|█████████▍| 47/50 [01:40<00:08,  2.92s/it]

s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_60min_base.parquet - 60min
Reading from s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_60min_base.parquet
Saving to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_60min_base_rsi.parquet


100%|██████████| 50/50 [01:43<00:00,  2.07s/it]


In [9]:
for path in tqdm(paths):
    name = path.split('/')[-1].split('.')[0].split('_')[-2]
    if 'base' in path:
        continue
    # if 'min' in name and int(name.replace('min', '')) < 100:  # Process files for <100 min
    print(path, '-', name)
    if 'min' in name:
        if int(name.replace('min', ''))<=10:
            windows = [7, 9, 11, 14, 17, 21]
            fields=['close', 'close_ema_2m', 'close_sma_2m']
        elif int(name.replace('min', ''))<=30:
            windows = [7, 9, 11, 14, 17, 21]
            fields=['close', 'close_ema_2m', 'close_sma_2m', 'close_ema_3m', 'close_sma_3m', 'close_ema_5m', 'close_sma_5m']
        else:
            windows = [7, 9, 11, 14, 17, 21]
            fields=['close', 'close_ema_5m', 'close_sma_5m', 'close_ema_10m', 'close_sma_10m', 'close_ema_20m', 'close_sma_20m', 'close_ema_50m', 'close_sma_50m']
    elif 'D' in name:
        if int(name.replace('D', '')) < 100:
            windows = [5, 7, 9, 11, 14, 17, 21]
            fields=['close', 'close_ema_5m', 'close_sma_5m', 'close_ema_10m', 'close_sma_10m', 'close_ema_20m', 'close_sma_20m', 'close_ema_50m', 'close_sma_50m']
        else: 
            windows = [5, 7, 9, 11, 14, 17, 21]
            fields=['close', 'close_ema_5m', 'close_sma_5m', 'close_ema_10m', 'close_sma_10m', 'close_ema_20m', 'close_sma_20m', 'close_ema_50m', 'close_sma_50m']
    fields = [f.replace('close', 'close_diff') for f in fields]
    read_and_calculate_rsi(path, windows, fields)

  0%|          | 0/50 [00:00<?, ?it/s]

s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_diff.parquet - 100D
Reading from s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_diff.parquet
Saving to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_diff_rsi.parquet


  4%|▍         | 2/50 [00:01<00:25,  1.91it/s]

s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10D_diff.parquet - 10D
Reading from s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10D_diff.parquet
Saving to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10D_diff_rsi.parquet


  8%|▊         | 4/50 [00:02<00:26,  1.72it/s]

...

 92%|█████████▏| 46/50 [01:34<00:07,  1.77s/it]

s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_5min_diff.parquet - 5min
Reading from s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_5min_diff.parquet
Saving to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_5min_diff_rsi.parquet


 96%|█████████▌| 48/50 [01:41<00:04,  2.19s/it]

s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_60min_diff.parquet - 60min
Reading from s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_60min_diff.parquet
Saving to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_60min_diff_rsi.parquet


100%|██████████| 50/50 [01:43<00:00,  2.07s/it]
